Get data file

Define X = images and y = multi-classes labels

preprocessing

data split

## Starting basic model

In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import EarlyStopping



We will have to change input shape, kernel.
we should use different layers to reduce smoothly before flattening!


In [ ]:
#### 1. ARCHITECTURE
model = Sequential()
model.add(layers.Conv2D(16, (3,3), input_shape=(28, 28, 1), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Conv2D(32, (2,2), padding='same', activation="relu"))
model.add(layers.MaxPool2D(pool_size=(2,2))) 
model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu')) # intermediate layer
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
#### 2. COMPILATION
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
es = EarlyStopping(patience= , restore_best_weights=True)
#### 3. FIT
history = model.fit(X_train, y_train, 
          batch_size=16, 
          epochs=1000, 
          validation_split=0.3,
          callbacks=[es])

Remember to plot history!

In [ ]:
# evaluate on the test data
print(model.evaluate(X_test, y_cat_test, verbose=0))